In [ ]:
import pandas as pd
import sqlalchemy
from binance.client import Client 
from binance import BinanceSocketManager
import keys

In [ ]:
# USING TESTNET FOR NOT TRADING REAL MONEY!!
client = Client(keys.TESTNET_API_KEY, keys.TESTNET_SECRET_KEY)

# ONLY NECESSARY FOR USING THE TESTNET
client.API_URL = 'https://testnet.binance.vision/api'

bsm = BinanceSocketManager(client)
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')

In [ ]:
df = pd.read_sql('BTCUSDT', engine)
print(df)

In [ ]:
df.price.plot()
# in normal python script this is done slightly differently - need to import matplotlib.pyplot as plt and do plt.show()

In [ ]:
print(client.get_asset_balance('BTC'))

In [ ]:
# Trend following - if crypto rising by x% buy, sell at +/- 0.15%
def strategy(entry, lookback, qty, open_position=False):
    while True:
        lookbackperiod = df.iloc[-lookback:] # filter the df for the last defined timestamp
        cumulative_return = (lookbackperiod.price.pct_change() + 1).cumprod() - 1
        if not open_position:
            if cumulative_return[cumulative_return.last_valid_index()] > entry:
                order = client.create_order(symbol='BTCUSDT',
                                            SIDE='BUY',
                                            type='MARKET',
                                            quantity=qty)
                print(order)
                open_position=True
                break # DON'T OPEN ANY MORE POSITIONS ONCE YOU HAVE BOUGHT
            
    if open_position:
        while True:
            df = pd.read_sql('BTCUSDT', engine)
            sincebuy = df.loc[df.time > pd.to_datetime(order['transactTime'],
                                                       unit='ms')]
            if len(sincebuy) > 1:
                sincebuyreturn = (sincebuy.price.pct_chance() + 1).cumprod() - 1
                last_entry = sincebuyreturn[sincebuyreturn.last_valid_index()]
                if last_entry > 0.0015 or last_entry < -0.0015:
                    order = client.create_order(symbol='BTCUSDT',
                                            SIDE='SELL',
                                            type='MARKET',
                                            quantity=qty)
                    print(order)
                    break # MAKE SURE TO BREAK OR YOU WILL SELL AGAIN